<a href="https://colab.research.google.com/github/Alex-Ukraine/ml-selfedu/blob/main/RNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Рекуррентные нейронные сети(Recurrent Neural Network(RNN)).

сеть Хопфилда в 1982 г, сеть Джеффа Эмана в 1990 г

Архитектуры:

* Many to Many - для перевода текстов

* Many to One - для анализа эмоциональной окраски текста (на входе текст, на выходе категории: положительный, отрицательный, нейтральный)

* One to Many - для генерации описания изображений, когда на вход подаються карты признаков изображения, а на выходе получаем его описание (текст);

* One to One - относительно редкая архитерктура для выполнения нелинейных рекуррентных вычислений.

Модифицированный алгоритм модифицированных нейронных сетей называется 

Backpropagation Through Time (BPTT)

устойчивость нелинейных динамических систем

нейродинамика - обеспечение устойчивости НС в процессе работы

устойчивость по Ляпунову, прямой метод Ляпунова

Саймон Хайкин "Нейронные сети(полный курс)"

In [21]:
import numpy as np
import re

In [22]:
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer

from google.colab import files
files.upload()

Saving train_data_true to train_data_true (1)


{'train_data_true': b'\xef\xbb\xbf\xd0\x94\xd1\x83\xd0\xbc\xd0\xb0\xd0\xb9\xd1\x82\xd0\xb5 \xd0\xbf\xd0\xbe\xd0\xb7\xd0\xb8\xd1\x82\xd0\xb8\xd0\xb2\xd0\xbd\xd0\xbe \xd0\xb8 \xd0\xb2\xd0\xb5\xd1\x80\xd1\x8c\xd1\x82\xd0\xb5 \xd0\xb2 \xd1\x81\xd0\xb2\xd0\xbe\xd1\x8e \xd1\x81\xd0\xbf\xd0\xbe\xd1\x81\xd0\xbe\xd0\xb1\xd0\xbd\xd0\xbe\xd1\x81\xd1\x82\xd1\x8c \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8\xd0\xb3\xd0\xb0\xd1\x82\xd1\x8c \xd0\xbe\xd1\x82\xd0\xbb\xd0\xb8\xd1\x87\xd0\xbd\xd1\x8b\xd1\x85 \xd1\x80\xd0\xb5\xd0\xb7\xd1\x83\xd0\xbb\xd1\x8c\xd1\x82\xd0\xb0\xd1\x82\xd0\xbe\xd0\xb2. \r\n\xd0\x92\xd1\x8b \xe2\x80\x94 \xd0\xbb\xd1\x83\xd1\x87\xd1\x88\xd0\xb8\xd0\xb9 \xd0\xbe\xd1\x82\xd0\xb2\xd0\xb5\xd1\x82 \xd0\xbd\xd0\xb0 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xb1\xd0\xbb\xd0\xb5\xd0\xbc\xd1\x8b, \xd0\xba\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd1\x8b\xd0\xb5 \xd0\xb2\xd0\xbe\xd0\xb7\xd0\xbd\xd0\xb8\xd0\xba\xd0\xbb\xd0\xb8 \xd0\xb2 \xd0\xbf\xd0\xbe\xd0\xbd\xd0\xb5\xd0\xb4\xd0\xb5\xd0\xbb\xd1\x8c\xd0\xbd\xd0\xb8\

In [23]:
with open('train_data_true', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я]', '', text) # заменяем все символы кроме кириллицы на пустые символы

One-hot-encoding (OHE)

tf.keras.preprocessing.text.Tokenizer

* num_words - максимальное количество слов (символов), которое вернет Tokenizer (если элементов будет больше, то остануться наиболее повторяющиеся в тексте);
* filters - исключаемые из текста символы (по умолчанию, следующие: !-Ю-№;%?()*+,-.:;<=>?@[\]^_'{|}~\t\n\r);
* lower=True - автоматический перевод в нижний регистр для единообразия больших и малых символов;
* split='' - разделение слов по символу проблела;
* char_level=False - есть False, то текст делиться на слова, иначе - на символы.

num_characters=34 # 33 буквы + пробел
tokenizer=Tokenizer(num_words=num_characters, char_level=True)

In [24]:
num_characters=34 # 33 буквы + пробел
tokenizer=Tokenizer(num_words=num_characters, char_level=True) # токенизируем на уровне символов
tokenizer.fit_on_texts([text]) # формируем токены на основе частотности в нашем тексте
print(tokenizer.word_index)

{'о': 1, 'е': 2, 'т': 3, 'и': 4, 'а': 5, 'н': 6, 'с': 7, 'в': 8, 'р': 9, 'м': 10, 'л': 11, 'ь': 12, 'д': 13, 'п': 14, 'у': 15, 'ы': 16, 'з': 17, 'я': 18, 'б': 19, 'ч': 20, 'к': 21, 'й': 22, 'ж': 23, 'г': 24, 'ш': 25, 'х': 26, 'ю': 27, 'ц': 28, 'щ': 29, 'э': 30, 'ф': 31, 'ъ': 32}


In [25]:
inp_chars = 6
data = tokenizer.texts_to_matrix(text) # преобразуем исходный текст в массив OHE
n = data.shape[0] - inp_chars # так как мы предсказываем по трём символам - четвёртый

# list comprehentions
X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:] # предсказание следующего символа

In [26]:
print(data.shape)

(5394, 34)


In [27]:
model = Sequential()
# входной слой имеет такой формат, это тензор (batch_size, inp_chars, num_characters)
model.add(Input((inp_chars,
                 num_characters))) # при тренировке в рекурентные модели keras подаётся сразу вся последовательность 
model.add(SimpleRNN(128, activation='tanh')) # рекуррентный слой на 128 нейронов
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               20864     
_________________________________________________________________
dense_1 (Dense)              (None, 34)                4386      
Total params: 25,250
Trainable params: 25,250
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [29]:
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
169/169 [==============================] - 1s 3ms/step - loss: 3.0747 - accuracy: 0.1403
Epoch 2/100
169/169 [==============================] - 1s 3ms/step - loss: 2.7871 - accuracy: 0.2216
Epoch 3/100
169/169 [==============================] - 1s 3ms/step - loss: 2.6083 - accuracy: 0.2641
Epoch 4/100
169/169 [==============================] - 1s 3ms/step - loss: 2.4854 - accuracy: 0.2877
Epoch 5/100
169/169 [==============================] - 1s 3ms/step - loss: 2.3931 - accuracy: 0.3044
Epoch 6/100
169/169 [==============================] - 1s 3ms/step - loss: 2.3243 - accuracy: 0.3170
Epoch 7/100
169/169 [==============================] - 1s 3ms/step - loss: 2.2669 - accuracy: 0.3324
Epoch 8/100
169/169 [==============================] - 1s 4ms/step - loss: 2.2206 - accuracy: 0.3391
Epoch 9/100
169/169 [==============================] - 1s 4ms/step - loss: 2.1765 - accuracy: 0.3582
Epoch 10/100
169/169 [==============================] - 1s 4ms/step - loss: 2.1349 - accura

In [30]:
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-Encoding
        
        x = np.array(x)
        # формируем начальные символы по которым будем делать прогноз
        inp = x.reshape(1, inp_chars, num_characters)

        pred = model.predict(inp) # предсказываем OHE четвёртого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

        inp_str += d # дописываем строку

    return inp_str

In [41]:
res = buildPhrase("сильне")
print(res)

сильнееиживешьсчастливоесердцескажитеисделайтенечтопозит
